In [24]:
# !pip install selenium
# !pip install underthesea # work_tokenize, sentiment
# ! pip install transformers

    100% |████████████████████████████████| 1.4MB 2.9MB/s ta 0:00:01
    100% |████████████████████████████████| 675kB 2.4MB/s ta 0:00:01
    100% |████████████████████████████████| 890kB 2.6MB/s ta 0:00:011
    100% |████████████████████████████████| 2.9MB 3.4MB/s ta 0:00:011    64% |████████████████████▊           | 1.9MB 2.9MB/s eta 0:00:01
  Stored in directory: /home/tructt2/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [1]:
import urllib.request
from bs4 import BeautifulSoup # parse html
import re #regex
import csv
import os
import json
import pandas as pd
import urllib.request
from sklearn.externals import joblib #load, dump pkl
from underthesea import word_tokenize #word_tokenize of lines
import numpy as np
import transformers as ppb # load model BERT
from transformers import BertModel, BertTokenizer
import torch
from sklearn.model_selection import train_test_split
# scrap comment = selenium
from selenium import webdriver 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
# import requests

# Craw comment of product tiki, lazada

In [2]:


def load_url_selenium_lazada(url):
    # Selenium
    driver=webdriver.Chrome(executable_path='/usr/bin/chromedriver')
    print("Loading url=", url)
    driver.get(url)
    list_review = []
    # just craw 10 page
    x=0
    while x<10:
        try:
            #Get the review details here
            WebDriverWait(driver,5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,"div.item")))
        except:
            print('No has comment')
            break
        
        product_reviews = driver.find_elements_by_css_selector("[class='item']")
        # Get product review
        for product in product_reviews:
            review = product.find_element_by_css_selector("[class='content']").text
            if (review != "" or review.strip()):
                print(review, "\n")
                list_review.append(review)
        #Check for button next-pagination-item have disable attribute then jump from loop else click on the next button
        if len(driver.find_elements_by_css_selector("button.next-pagination-item.next[disabled]"))>0:
            break;
        else:
            button_next=WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "button.next-pagination-item.next")))
            driver.execute_script("arguments[0].click();", button_next)
            print("next page")
            time.sleep(2)
            x +=1
    driver.close()
    return list_review

def load_url_selenium_tiki(url):
    driver=webdriver.Chrome(executable_path='/usr/bin/chromedriver')
    print("Loading url=", url)
    driver.get(url)
    list_review = []
    # just craw 10 page
    x=0
    while x<10:
        try:
          
            #Get the review details here
            WebDriverWait(driver,5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,"div.review-comment")))
        except :
            print('Not has comment!')
            break
        product_reviews = driver.find_elements_by_css_selector("[class='review-comment']")
        # Get product review
        for product in product_reviews:
            review = product.find_element_by_css_selector("[class='review-comment__content']").text
            if (review != "" or review.strip()):
                print(review, "\n")
                list_review.append(review)
        #Check for button next-pagination-item have disable attribute then jump from loop else click on the next button
        try:
            #driver.find_element_by_xpath("//li[@class='btn next']/a").click()
            button_next=WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "[class = 'btn next']")))
            driver.execute_script("arguments[0].click();", button_next)
            print("next page")
            time.sleep(2)
            x +=1
        except (TimeoutException, WebDriverException) as e:
            print('Load several page!')
            break
    driver.close()
    return list_review



# Standard data, tokenizer

In [4]:

def standardize_data(row):
    # remove stopword
    # Remove . ? , at index final
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Remove all . , " ... in sentences
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")

    row = row.strip()
    return row

# Tokenizer
def tokenizer(row):
    return word_tokenize(row, format="text")

def analyze(result):
    bad = np.count_nonzero(result)
    good = len(result) - bad
    print("No of bad and neutral comments = ", bad)
    print("No of good comments = ", good)

    if good>bad:
        return "Good! You can buy it!"
    else:
        return "Bad! Please check it carefully!"


# Processing data

In [5]:
def processing_data(data):
    # 1. Standardize data
    data_frame = pd.DataFrame(data)
    print('data frame:', data_frame)
    data_frame[0] = data_frame[0].apply(standardize_data)

    # 2. Tokenizer
    data_frame[0] = data_frame[0].apply(tokenizer)

    # 3. Embedding
    X_val = data_frame[0]
    return X_val


# Load Pretrain model BERT

In [6]:
def load_pretrainModel(data):
    
    '''
    Load pretrain model/ tokenizers
    Return : features
    '''
    model = BertModel.from_pretrained('bert-base-uncased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    #encode lines
    tokenized = data.apply((lambda x: tokenizer.encode(x, add_special_tokens = True)))

    # get lenght max of tokenized
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
    print('max len:', max_len)

    # if lenght of tokenized not equal max_len , so padding value 0
    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    print('padded:', padded[1])
    print('len padded:', padded.shape)

    #get attention mask ( 0: not has word, 1: has word)
    attention_mask = np.where(padded ==0, 0,1)
    print('attention mask:', attention_mask[1])

    # Convert input to tensor
    padded = torch.tensor(padded)
    attention_mask = torch.tensor(attention_mask)


    # Load model
    with torch.no_grad():
        last_hidden_states = model(padded, attention_mask =attention_mask)
    #     print('last hidden states:', last_hidden_states)

    features = last_hidden_states[0][:,0,:].numpy()
    print('features:', features)
    
    return features


# Predict

In [8]:
def predict(url):
    # 1. Load URL and print comments
    if url== "":
        url = "https://tiki.vn/dien-thoai-samsung-galaxy-m31-128gb-6gb-hang-chinh-hang-p58259141.html"
    data = load_url_selenium_lazada(url)
#     data = load_url_selenium_tiki(url)
    data = processing_data(data)
    features = load_pretrainModel(data)
    # 2. Load weights
    model = joblib.load('save_model.pkl')
    # 3. Result
    result = model.predict(features)
    print(result)
    print(analyze(result))
# predict(url ='https://tiki.vn/dien-thoai-samsung-galaxy-s20-plus-hang-chinh-hang-p48886153.html?src=search&2hi=0&keyword=s20&src=mega-menu')
predict(url = 'https://www.lazada.vn/products/iphone-8-plus-chinh-hang-vna-moi-100-chua-kich-hoat-chua-qua-su-dung-bao-hanh-12-thang-tai-ttbh-apple-tra-gop-lai-suat-0-qua-the-tin-dung-man-hinh-retina-hd-55-inch-3d-touch-chip-a11-ios11-i757986604-s1985088475.html?spm=a2o4n.searchlistcategory.list.4.46d0bdd5OzWEVE&search=1')

Loading url= https://www.lazada.vn/products/iphone-8-plus-chinh-hang-vna-moi-100-chua-kich-hoat-chua-qua-su-dung-bao-hanh-12-thang-tai-ttbh-apple-tra-gop-lai-suat-0-qua-the-tin-dung-man-hinh-retina-hd-55-inch-3d-touch-chip-a11-ios11-i757986604-s1985088475.html?spm=a2o4n.searchlistcategory.list.4.46d0bdd5OzWEVE&search=1
5 sao; ổn nha nhưng có điều hàng giá trị cao mà Lazada vẫn lấy tiền vận chuyển 9900d, thật hài vãi. Mới nhận hàng nhưng thấy khá ổn, nên mua nha mọi người. à Chú giao hàng khá dễ thương ok 

Hàng chuẩn, nguyên Seal, SX 2020 Vs ios 13. Nhưng trong hộp không có cáp chuyển đổi 3.5 cho tai nghe nên chỉ cho 4 sao. Viết đánh giá này bằng chính iphone 8plus 

Mới nhận sp Iphone 8 plus 64GB, gồm 1 máy, 1 cáp sạc, 1 củ sạc, 1 tai nghe, 1 kim lấy sim, hướng dẫn... nhưng thiếu 1 cáp chuyển đổi nha NBH Apple! (K thể đăng ảnh) 

Hàng chuẩn, IMEI máy và vỏ trùng nhau. Lần đầu dùng Iphone nên thấy hơi khó dùng. Chạy Giá sale 12.12 so với mua offline rất rẻ. Hài lòng 

K có giác chia 



next page
Mua trả góp nhưng cuối cùng Lazada làm ăn quá cẩu thả, không chuyển hồ sơ trả góp của khách qua ngân hàng, cuối cùng khách hàng là người thiệt thòi, không còn tin tưởng được Lazada 

Hàng đóng gói rất kỹ, shipper lịch sự gọi trước khi đến.
Điện thoại mã VN chuẩn, nguyên seal và máy rất đẹp. Giá tốt  

Giao hàng đúng hẹn. Đóng gói không hài lòng. Điện thoại mới, không tì vết. Giá hợp lý (nhưng ko phải đã là rẻ).  Mong dịch vụ bảo hành tốt. 

Mình ở ngoài Hà Nội. Đặt sáng chiều hôm trước thì sáng sớm hôm sau có hàng luôn. Hàng đúng mô tả: chưa active - VN/A. Rất ưng cái bụng vì mua được hàng mới mà giá rẻ. 

Điện thoại nhận được còn nguyên seal.
sau khi thanh toán mở ra kiểm tra thì máy hoạt động tốt, nguyên bản chưa active.  

next page
Sản phẩm fullbox, chưa active. Sau khi activate, ngày bảo hành trên web của apple cập nhật đúng ngày. Có điều giao hàng hơi chậm, sau 4 ngày mơi giao. Giá khá tốt: rẻ hơn tgdđ và fptshop 4tr, lúc mua: 19.990.000 

Sản 